# Ejercicio 1 (Parte en R).

Iniciamos cargando los datos que vamos a utilizar:

In [1]:
cov_data = read.csv('covid.csv')
head(cov_data, 3)

,X,SEXO,OBESIDAD,DIABETES,EPOC,ASMA,CARDIOVASCULAR,RENAL.CRONICA,INMUNOSUPRESION,HIPERTENSION,TIPO.PACIENTE,EDAD
,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>
1,0,MUJER,NO,NO,NO,NO,NO,NO,NO,NO,AMBULATORIO,51
2,1,MUJER,NO,NO,NO,NO,NO,NO,NO,NO,AMBULATORIO,37
3,2,HOMBRE,NO,NO,NO,NO,NO,NO,NO,NO,AMBULATORIO,18


Eliminamos la columna X

In [2]:
cov_data = subset(cov_data, select=-c(X))

In [3]:
cov_data <- sapply(cov_data, as.numeric)
cov_data <- cov_data-1
cov_data = as.data.frame(cov_data)

In [4]:
head(cov_data, 3)

,SEXO,OBESIDAD,DIABETES,EPOC,ASMA,CARDIOVASCULAR,RENAL.CRONICA,INMUNOSUPRESION,HIPERTENSION,TIPO.PACIENTE,EDAD
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,0,0,0,0,0,0,0,0,50
2,1,0,0,0,0,0,0,0,0,0,36
3,0,0,0,0,0,0,0,0,0,0,17


Tomamos una muestra aleatoria, de lo contrario los algoritmos de selección de variables no añaden o quitan ninguna de las covariables. 

In [5]:
test <- sample(nrow(cov_data), 15000)
test_data <- cov_data[test,]

### a) Utilizar el AIC con forward, backward y stepwise

**Forward**

Empezamos con el forward. Para ejecutar este algoritmo, tenemos que iniciar con el modelo vacío, así que usaremos TIPO.PACIENTE~1

In [6]:
library("MASS")

In [7]:
empty.model <- lm(TIPO.PACIENTE ~ 1, data=test_data)
horizonte <- formula(~., data=test_data)

In [8]:
modforw <- stepAIC(empty.model, trace=TRUE, direction="forward", 
                   scope=list(lower = TIPO.PACIENTE ~ EDAD, upper = TIPO.PACIENTE ~ .),
                   k = log(100))

Start:  AIC=-23696.79
TIPO.PACIENTE ~ 1



In [9]:
summary(modforw)


Call:
lm(formula = TIPO.PACIENTE ~ 1, data = test_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.2901 -0.2901 -0.2901  0.7099  0.7099 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.290133   0.003706    78.3   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4538 on 14999 degrees of freedom


Con este método no llegamos a nada, posiblemente porque tenemos muchas observaciones contra pocas covariables y no se encontró ninguna lo suficientemente significativa para ser añadida al modelo en el mismo paso. 

Lo prové con otros tamaños de muestra, pero no logré que cambiara el resultado (si fue error mío, me gustaría si me pudiera comentar qué fue).

**Backward**

Con este método iniciamos con el modelo completo para ir eliminando covariables, por lo que se usa TIPO.PACIENTE~.

In [10]:
full.model <- lm(TIPO.PACIENTE ~ ., data=test_data)
backwards_model <- stepAIC(full.model, trace=TRUE, direction="backward")

Start:  AIC=-26920.84
TIPO.PACIENTE ~ SEXO + OBESIDAD + DIABETES + EPOC + ASMA + CARDIOVASCULAR + 
    RENAL.CRONICA + INMUNOSUPRESION + HIPERTENSION + EDAD

                  Df Sum of Sq    RSS    AIC
- ASMA             1     0.000 2488.9 -26923
- CARDIOVASCULAR   1     0.180 2489.1 -26922
<none>                         2488.9 -26921
- EPOC             1     0.544 2489.5 -26920
- HIPERTENSION     1     2.373 2491.3 -26908
- OBESIDAD         1     3.212 2492.2 -26904
- INMUNOSUPRESION  1     4.710 2493.7 -26894
- RENAL.CRONICA    1     8.242 2497.2 -26873
- SEXO             1    29.596 2518.6 -26746
- DIABETES         1    38.007 2527.0 -26696
- EDAD             1   264.882 2753.8 -25406

Step:  AIC=-26922.84
TIPO.PACIENTE ~ SEXO + OBESIDAD + DIABETES + EPOC + CARDIOVASCULAR + 
    RENAL.CRONICA + INMUNOSUPRESION + HIPERTENSION + EDAD

                  Df Sum of Sq    RSS    AIC
- CARDIOVASCULAR   1     0.180 2489.1 -26924
<none>                         2488.9 -26923
- EPOC          

In [11]:
summary(backwards_model)


Call:
lm(formula = TIPO.PACIENTE ~ SEXO + OBESIDAD + DIABETES + EPOC + 
    RENAL.CRONICA + INMUNOSUPRESION + HIPERTENSION + EDAD, data = test_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.0336 -0.2747 -0.1273  0.3569  1.2227 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -0.1241417  0.0108028 -11.492  < 2e-16 ***
SEXO            -0.0893078  0.0066809 -13.368  < 2e-16 ***
OBESIDAD         0.0374789  0.0084860   4.417 1.01e-05 ***
DIABETES         0.1501218  0.0098862  15.185  < 2e-16 ***
EPOC             0.0482891  0.0256840   1.880 0.060111 .  
RENAL.CRONICA    0.1738053  0.0243487   7.138 9.89e-13 ***
INMUNOSUPRESION  0.1663981  0.0309102   5.383 7.42e-08 ***
HIPERTENSION     0.0367383  0.0094663   3.881 0.000105 ***
EDAD             0.0092104  0.0002297  40.097  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4075 on 14991 degrees of freedom
Multiple R-squared:  0.

Vemos que en este caso se eliminan el ASMA y CARDIOVASCULAR. 

**Stepwise**

Se permite añadir y remover covariables. 
En este caso, si iniciamos desde el modelo vacío llegamos a lo mismo que con forward, es decir, no se añaden variables, por lo que inicio desde el modelo lleno.

In [12]:
modboth <- stepAIC(full.model, trace=TRUE, direction="both", scope=horizonte)

Start:  AIC=-26920.84
TIPO.PACIENTE ~ SEXO + OBESIDAD + DIABETES + EPOC + ASMA + CARDIOVASCULAR + 
    RENAL.CRONICA + INMUNOSUPRESION + HIPERTENSION + EDAD

                  Df Sum of Sq    RSS    AIC
- ASMA             1     0.000 2488.9 -26923
- CARDIOVASCULAR   1     0.180 2489.1 -26922
<none>                         2488.9 -26921
- EPOC             1     0.544 2489.5 -26920
- HIPERTENSION     1     2.373 2491.3 -26908
- OBESIDAD         1     3.212 2492.2 -26904
- INMUNOSUPRESION  1     4.710 2493.7 -26894
- RENAL.CRONICA    1     8.242 2497.2 -26873
- SEXO             1    29.596 2518.6 -26746
- DIABETES         1    38.007 2527.0 -26696
- EDAD             1   264.882 2753.8 -25406

Step:  AIC=-26922.84
TIPO.PACIENTE ~ SEXO + OBESIDAD + DIABETES + EPOC + CARDIOVASCULAR + 
    RENAL.CRONICA + INMUNOSUPRESION + HIPERTENSION + EDAD

                  Df Sum of Sq    RSS    AIC
- CARDIOVASCULAR   1     0.180 2489.1 -26924
<none>                         2488.9 -26923
- EPOC          

In [13]:
summary(modboth)


Call:
lm(formula = TIPO.PACIENTE ~ SEXO + OBESIDAD + DIABETES + EPOC + 
    RENAL.CRONICA + INMUNOSUPRESION + HIPERTENSION + EDAD, data = test_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.0336 -0.2747 -0.1273  0.3569  1.2227 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -0.1241417  0.0108028 -11.492  < 2e-16 ***
SEXO            -0.0893078  0.0066809 -13.368  < 2e-16 ***
OBESIDAD         0.0374789  0.0084860   4.417 1.01e-05 ***
DIABETES         0.1501218  0.0098862  15.185  < 2e-16 ***
EPOC             0.0482891  0.0256840   1.880 0.060111 .  
RENAL.CRONICA    0.1738053  0.0243487   7.138 9.89e-13 ***
INMUNOSUPRESION  0.1663981  0.0309102   5.383 7.42e-08 ***
HIPERTENSION     0.0367383  0.0094663   3.881 0.000105 ***
EDAD             0.0092104  0.0002297  40.097  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4075 on 14991 degrees of freedom
Multiple R-squared:  0.

En este caso, el resultado fue el mismo que con backwards. Por lo mismo, conjeturo que por las características de los datos tal vez estos algoritmos no sean el método más adecuado para hacer la selección de covariables.

### b) Utilizar los criterios de BIC y Mallow Cp

In [14]:
library(leaps)

In [15]:
all <-regsubsets(TIPO.PACIENTE ~ ., data=test_data, nbest=1)

In [16]:
info<-summary(all)
cbind(info$which, round(cbind(rsq=info$rsq,adjr2=info$adjr2, cp=info$cp, bic=info$bic, rss=info$rss), 3))

,(Intercept),SEXO,OBESIDAD,DIABETES,EPOC,ASMA,CARDIOVASCULAR,RENAL.CRONICA,INMUNOSUPRESION,HIPERTENSION,EDAD,rsq,adjr2,cp,bic,rss
1,1,0,0,0,0,0,0,0,0,0,1,0.159,0.159,655.577,-2573.392,2598.972
2,1,0,0,1,0,0,0,0,0,0,1,0.177,0.177,317.039,-2893.741,2542.424
3,1,1,0,1,0,0,0,0,0,0,1,0.186,0.186,144.820,-3055.783,2513.495
4,1,1,0,1,0,0,0,1,0,0,1,0.190,0.190,74.426,-3118.079,2501.474
5,1,1,0,1,0,0,0,1,1,0,1,0.192,0.192,45.023,-3139.765,2496.260
6,1,1,1,1,0,0,0,1,1,0,1,0.193,0.193,23.167,-3153.971,2492.298
7,1,1,1,1,0,0,0,1,1,1,1,0.194,0.194,9.617,-3159.904,2489.716
8,1,1,1,1,1,0,0,1,1,1,1,0.194,0.194,8.083,-3153.825,2489.129


**BIC**: El modelo con el menor BIC es el 7, en donde se eliminan ASMA, CARDIOVASCULAR y EPOC.

**Mallow Cp**: El modelo con el Mallow Cp más pequeño es el 8, en donde se eliminan ASMA y CARDIOVASCULAR.